In [ ]:
# The packages needed for this project of prediting flight delays
#from pyspark.sql import SQLContext
import pandas as pd
import numpy as np
#import pyspark.sql as sparksql
import findspark
from matplotlib import pyplot as plt
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
import seaborn as sns
from pyspark.sql.functions import isnan
from pyspark.sql.functions import col, sum
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basics').getOrCreate()

In [ ]:
# This is read the data from csv and create a data frame
df = spark.read.csv('DelayedFlight.csv',header=True,inferSchema=True)

In [ ]:
#This show the top 5 rows in the data set (just to see the type of data if its numerical or txt value)
df.show(5)

In [ ]:
#show the data type in the dataset
df.printSchema()
print(df.head())

In [ ]:
#This show the number of rows in the dataset (raw data)
df.count()

In [ ]:
#Describe Operation using here is use to calculate the summary of all numerical columns in the data frame,hence the ext code was choose only few to explore the data set 
df.describe().show()

In [ ]:
# To show the summary statistics of target attribute is "ArrDelay" and other variables correlated attributes.
df.describe('ArrDelay', 'DepDelay', 'CarrierDelay', 'LateAircraftDelay', 'NASDelay', 'WeatherDelay').show()

In [ ]:
#Filter the target variables rows that contains values (NotNull)
df.filter(df.ArrDelay.isNull()).count() 
df.filter(df.ArrDelay.isNotNull()).count()

In [ ]:
#This shows the number of missing value in the target variable "ArrDelay" 
df.filter((df['ArrDelay'] == '') | df ['ArrDelay'].isNull() | isnan (df['ArrDelay'])) .count()

In [ ]:
#This code shows the summary of all the missing values in each field
from pyspark.sql.functions import lit
rows = df.count() 
summary = df.describe().filter(col("summary") == "count") 
summary.select(*((lit(rows)-col(c)).alias(c) for c in df.columns)).show()

In [ ]:
# This code is show the name of the fields or columns in the dataset (raw data)
len(df.columns), df.columns

In [ ]:
# This code is show the total number of columns in the dataset
len(df.columns)

In [ ]:
#Collection of the target variables
df.corr('ArrDelay','DepDelay')

In [ ]:
# Selecting delay_col and assign to the target variables
delay_col= df.select('ArrDelay')
delay_col.show(5)

In [ ]:
#Adding another columns in the data set and show only 5 rows
df.withColumn('Delay_times_5',df['ArrDelay']*5).show(5)


In [ ]:
# Run the dataframe and new output missing the new column
df.show(5)

In [ ]:
#Using filter within the dataframe
df.filter("ArrDelay < 15").show()

In [ ]:
#Arrival delay more than 15 minutes and selction from the carrier and flight number 
#also how long the departure delay since the arrival flight delay
df.filter("ArrDelay > 15").select('DepDelay','FlightNum', 'UniqueCarrier').show()

In [ ]:
#Sample of the advance filter - values between 5 and 10 minutes delay
df.filter('ArrDelay <= 5' and 'ArrDelay >=10')\
  .select('FlightNum','DepDelay').distinct().count()

In [ ]:
df.groupBy('UniqueCarrier').mean().show(5)

In [ ]:
# To simplify things, let's split this into two steps. First, let's create a variable then order by age.
# Careful when using show()! Otherwise the variable type will change and you won't be able to order it. 
group_UniqueCarrier_df = df.groupBy('UniqueCarrier').mean()

# Note that we have to use 'avg(age)' instead of age. Why? Because when you use mean(), it changes the feature's name (as you can see below).
print("Sorted by ArrDelay")
group_UniqueCarrier_df.orderBy('avg(ArrDelay)').show()

# Let's see what this looks like in one line.
print("Sorted by DepDelay")
df.groupBy('UniqueCarrier').mean().orderBy('avg(DepDelay)').show()


In [ ]:
group_UniqueCarrier_df = df.groupBy('ArrDelay').mean()

In [ ]:
print("Sorted by UniqueCarrier")
group_UniqueCarrier_df.orderBy('avg(ArrDelay)').show(2)

In [ ]:
print("Sorted by UniqueCarrier")
df.groupBy('UniqueCarrier').mean().orderBy('avg(DepDelay)').show(2)

In [ ]:
df = df.na.drop(subset="ArrDelay")
df.show()
print("Total data points:", df.count())

In [ ]:
#Find mean average
from pyspark.sql.functions import mean
mean_ArrDelay = df.select(mean('ArrDelay')).show()
mean_ArrDelay= df.select(mean('ArrDelay')).collect()
print(mean_ArrDelay)
mean_ArrDelay = mean_ArrDelay[0]
print(mean_ArrDelay)
mean_ArrDelay= mean_ArrDelay[0]
print(mean_ArrDelay)

In [ ]:
#fill missing value using mean average
df = df.na.fill(mean_ArrDelay, subset=['ArrDelay'])
df=df.show()

In [ ]:
df.na.drop().show()

# Let's see how many rows of data we have now. 
print("Total data points:", df.count())